In [59]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
from glob import glob
import pickle
import pandas as pd

In [60]:
haar=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
mean=pickle.load(open('dataset/mean_preprocessing.pickle','rb'))
modle_svm=pickle.load(open('dataset/model_svm.pickle','rb'))
modle_pca=pickle.load(open('dataset/pca_50.pickle','rb'))
print('model load sucessfully')

model load sucessfully


In [61]:

gender_pre=['male','female']
font=cv2.FONT_HERSHEY_SIMPLEX

In [130]:

#convert into gray scale
def pipeline_model(img,color='rgb'):
    if color =='bgr':
        gray_cv=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    else:
        gray_cv=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #crop face
    face=haar.detectMultiScale(gray_cv,1.3,3)
    for x,y,w,h in face:
        cv2.rectangle(img,(x,y),(x+h,y+h),(0,255,0),2)#drawing rectangle
        roi=gray_cv[y:y+h,x:x+w] #croping image
        # normalization
        roi=roi/255.0
        # resize iamge 100x100
        if roi.shape[1]>30:
            roi_resize=cv2.resize(roi,(100,100),cv2.INTER_AREA)
        else:
            roi_resize=cv2.resize(roi,(100,100),cv2.INTER_CUBIC)
        # FLATTENING
        roi_reshape=roi_resize.reshape(1,10000)
        # subtract with mean
        roi_mean=roi_reshape-mean
        # get eigen image
        eigen_image=modle_pca.transform(roi_mean)
        # pass to ml model
        result=modle_svm.predict_proba(eigen_image)[0]
        # prediction and score
        predict=result.argmax()
        score=result[predict]
        # step 11
        text="%s :%0.2f"%(gender_pre[predict],score)
        cv2.putText(img,text,(x,y),font,1,(0,255,0),2)
    return img


In [131]:
#read image
test_data_path='bb.jpg'
img=Image.open(test_data_path) #accepted pic in RGB
color='bgr'
# convert into array
img=np.array(img)
# pass to pipeline model
img=pipeline_model(img)
cv2.imshow('gender',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [103]:
# apply pipeline model to video
cap=cv2.VideoCapture('vid3.mp4')

while True:
    ret,frame=cap.read()
    
    if ret==False:
        break
    frame=pipeline_model(frame,color='bgr')
    cv2.imshow('gender_detection',frame)
    if cv2.waitKey(20)==27:
        break
cv2.destroyAllWindows()
cap.release()